In [1]:
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy swifter
!pip install flow_vis

     |████████████████████████████████| 647 kB 10.3 MB/s 
     |████████████████████████████████| 280 kB 77.3 MB/s 
     |████████████████████████████████| 136 kB 85.6 MB/s 


In [2]:
import os
import glob
import numpy as np
import random
import pandas as pd
from time import sleep
import cv2 
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
import mediapy as media
from google.colab.patches import cv2_imshow
import mediapy as media
import flow_vis
import swifter
from sklearn.metrics import classification_report


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Create target directory & all intermediate directories if don't exists
def create_dir(dirName):
  try:
      os.makedirs(dirName)    
      print("Directory " , dirName ,  " Created ")
  except FileExistsError:
      print("Directory " , dirName ,  " already exists")  
      

In [5]:
PATH_DATA = '/content/drive/MyDrive/ucf_experiments/data/Peliculas/'

PATH_TEST_RESIZED = PATH_DATA + 'resized/'
create_dir(PATH_TEST_RESIZED)

Directory  /content/drive/MyDrive/ucf_experiments/data/Peliculas/resized/  already exists


In [6]:
PATH_CROP = PATH_DATA + 'data_crop/'
PATH_BLUR = PATH_DATA + 'data_blur/'

PATH_CROP_TEST = PATH_CROP + 'test/'
PATH_BLUR_TEST = PATH_BLUR + 'test/'

PATH_CHECKPOINTS = PATH_DATA + 'data_checkpoints/'
PATH_CHECKPOINTS_TEST = PATH_CHECKPOINTS + 'test/'

create_dir(PATH_CROP)
create_dir(PATH_BLUR)
create_dir(PATH_CROP_TEST)
create_dir(PATH_BLUR_TEST)

create_dir(PATH_CHECKPOINTS)
create_dir(PATH_CHECKPOINTS_TEST)


# Pós Crop
PATH_CROP_TEST_RESIZED = PATH_CROP + 'test_resized/'
create_dir(PATH_CROP_TEST_RESIZED)

PATH_CHECKPOINTS_TEST_RESIZED = PATH_CHECKPOINTS + 'test_resized/'
create_dir(PATH_CHECKPOINTS_TEST_RESIZED)

Directory  /content/drive/MyDrive/ucf_experiments/data/Peliculas/data_crop/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/data/Peliculas/data_blur/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/data/Peliculas/data_crop/test/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/data/Peliculas/data_blur/test/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/data/Peliculas/data_checkpoints/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/data/Peliculas/data_checkpoints/test/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/data/Peliculas/data_crop/test_resized/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/data/Peliculas/data_checkpoints/test_resized/  already exists


# Geração de Labels

In [ ]:
len(os.listdir(PATH_DATA + 'data/fights/'))

100

In [ ]:
movies_list1 = glob.glob(PATH_DATA + 'data/fights/*.avi')
df_videos1 = pd.DataFrame()
df_videos1 = pd.DataFrame({'video_path':movies_list1})
df_videos1['label'] = 1
df_videos1.head(3)

,video_path,label
0,/content/drive/MyDrive/ucf_experiments/data/Pe...,1
1,/content/drive/MyDrive/ucf_experiments/data/Pe...,1
2,/content/drive/MyDrive/ucf_experiments/data/Pe...,1


In [ ]:
len(os.listdir(PATH_DATA + 'data/noFights/'))

101

In [ ]:
movies_list2 = glob.glob(PATH_DATA + 'data/noFights/*.mpg')
df_videos2 = pd.DataFrame()
df_videos2 = pd.DataFrame({'video_path':movies_list2})
df_videos2['label'] = 0
df_videos2.head(3)

,video_path,label
0,/content/drive/MyDrive/ucf_experiments/data/Pe...,0
1,/content/drive/MyDrive/ucf_experiments/data/Pe...,0
2,/content/drive/MyDrive/ucf_experiments/data/Pe...,0


In [ ]:
df_videos = pd.concat([df_videos1, df_videos2], axis=0).reset_index(drop=True)
print(df_videos.shape)
print(df_videos.label.value_counts())
display(df_videos.tail(3))

(201, 2)
0    101
1    100
Name: label, dtype: int64


,video_path,label
198,/content/drive/MyDrive/ucf_experiments/data/Pe...,0
199,/content/drive/MyDrive/ucf_experiments/data/Pe...,0
200,/content/drive/MyDrive/ucf_experiments/data/Pe...,0


# Pre Processing

## Resized Videos

In [ ]:
# Normalização do dimensionamento do vídeo
def test_video_normalize(video_numpy):
  with tf.device('/gpu:0'):
    augs = ['resized']
    video_tensor = tf.constant(video_numpy)
    resized = tf.image.resize(video_numpy, size=(224, 224))
    stacked  = tf.stack([resized])

    return augs, stacked


def test_normalization(row):

  video_name = ''
  if(row['video_path'].split('.')[-1] == 'avi'):
    video_name = str(row['video_path'].split('/')[-1].replace('.avi', '')) +  '_' + str(row['label'])
    
  elif(row['video_path'].split('.')[-1] == 'mpg'):
    video_name = str(row['video_path'].split('/')[-1].replace('.mpg', '')) +  '_' + str(row['label'])

  raw_video = media.read_video(row['video_path'])
  raw_video = media.to_float01(raw_video)

  augs, videos = test_video_normalize(raw_video)
  
  for idx, name in enumerate(augs):
    aug_video_name = video_name + '_' + name + '.mp4'
    aug_path = os.path.join(PATH_TEST_RESIZED, aug_video_name)

    media.write_video(aug_path, videos[idx].numpy())

    row[name] = aug_path

  return row

In [ ]:
x_test_resized = df_videos.swifter.apply(test_normalization, axis=1)

Pandas Apply:   0%|          | 0/201 [00:00<?, ?it/s]

In [ ]:
from pathlib import Path
paths = sorted(Path(PATH_TEST_RESIZED).iterdir(), key=os.path.getmtime, reverse=True)
len(paths)

201

In [ ]:
print(len(glob.glob(PATH_TEST_RESIZED + '*.mp4')))

201


In [ ]:
x_test_resized.head(3)

,video_path,label,resized
0,/content/drive/MyDrive/ucf_experiments/data/Pe...,1,/content/drive/MyDrive/ucf_experiments/data/Pe...
1,/content/drive/MyDrive/ucf_experiments/data/Pe...,1,/content/drive/MyDrive/ucf_experiments/data/Pe...
2,/content/drive/MyDrive/ucf_experiments/data/Pe...,1,/content/drive/MyDrive/ucf_experiments/data/Pe...


In [ ]:
x_test_resized.to_csv(
    PATH_TEST_RESIZED + 'x_test_resized.csv'
)

In [ ]:
from pathlib import Path
paths = sorted(Path(PATH_TEST_RESIZED).iterdir(), key=os.path.getmtime, reverse=True)
len(paths)

202

## Optical Flow


In [ ]:
#videos_test = pd.read_csv(PATH_TEST_RESIZED + "x_test_resized.csv")
print(PATH_DATA)

/content/drive/MyDrive/ucf_experiments/data/Peliculas/


In [ ]:

def flow_to_color(flow, hsv):
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

def getOpticalFlow(video, return_rgb_flow=True):
    """Calculate dense optical flow of input video
    Args:
        video: the input video with shape of [frames,height,width,channel]. dtype=np.array
    Returns:
        flows_x: the optical flow at x-axis, with the shape of [frames,height,width,channel]
        flows_y: the optical flow at y-axis, with the shape of [frames,height,width,channel]
    """
    # initialize the list of optical flows
    
    gray_video = []
    for i in range(len(video)):
        img = cv2.cvtColor(video[i], cv2.COLOR_RGB2GRAY)
        gray_video.append(np.reshape(img,(224,224,1)))

    flows = []
    rgb_flows = []
    for i in range(0,len(video)-1):
        # calculate optical flow between each pair of frames


        flow = cv2.calcOpticalFlowFarneback(gray_video[i], gray_video[i+1], None, 0.5, 3, 15, 3, 5, 1.2, cv2.OPTFLOW_FARNEBACK_GAUSSIAN)
        # subtract the mean in order to eliminate the movement of camera
        flow[..., 0] -= np.mean(flow[..., 0])
        flow[..., 1] -= np.mean(flow[..., 1])
        # normalize each component in optical flow
        flow[..., 0] = cv2.normalize(flow[..., 0],None,0,255,cv2.NORM_MINMAX)
        flow[..., 1] = cv2.normalize(flow[..., 1],None,0,255,cv2.NORM_MINMAX)
        # Add into list 

        gss_app = cv2.GaussianBlur(flow,(5,5),1)

        if return_rgb_flow:
          flow_color = flow_vis.flow_to_color(gss_app, convert_to_bgr=False)
          rgb_flows.append(flow_color)

        flows.append(gss_app)
        
    # Padding the last frame as empty array
    flows.append(np.zeros((224,224,2)))

    flows = normalize(flows)
      
    return (np.array(flows, dtype=np.float32), np.array(rgb_flows))

def normalize(data):
    mean = np.mean(data)
    std = np.std(data)
    return (data-mean) / std

def dynamic_crop(opt_flows, rgb_frames):

    rgb_video = rgb_frames.copy()
    # sum of optical flow magnitude of individual frame
    magnitude = np.sum(opt_flows, axis=0)
    # filter slight noise by threshold 
    thresh = np.mean(magnitude)
    magnitude[magnitude<thresh] = 0
    # calculate center of gravity of magnitude map and adding 0.001 to avoid empty value
    x_pdf = np.sum(magnitude, axis=1) + 0.001
    y_pdf = np.sum(magnitude, axis=0) + 0.001
    # normalize PDF of x and y so that the sum of probs = 1
    x_pdf /= np.sum(x_pdf)
    y_pdf /= np.sum(y_pdf)

    x_pdf = x_pdf.sum(axis=1)
    y_pdf = y_pdf.sum(axis=1)

    # randomly choose some candidates for x and y 
    x_points = np.random.choice(a=np.arange(224), size=10, replace=True, p=x_pdf)
    y_points = np.random.choice(a=np.arange(224), size=10, replace=True, p=y_pdf)
    # get the mean of x and y coordinates for better robustness
    x = int(np.mean(x_points))
    y = int(np.mean(y_points))
    # avoid to beyond boundaries of array
    x = max(56,min(x,167))
    y = max(56,min(y,167))
    # get cropped video

    cropped = rgb_video[:,x-56:x+56,y-56:y+56,:].copy()

    w = 112
    h = 112

    start_point = (int(x-w/2), int(y-h/2))
    end_point = (int(x+w/2), int(y+h/2))

    color = (255,255,255)
    thickness = -1
    blurred_video = np.zeros_like(rgb_video)

    for idx, frame in enumerate(rgb_video):
      blurred_img = cv2.GaussianBlur(frame, (21, 21), 0)
      mask = np.zeros((frame.shape[0], frame.shape[1], 3), dtype=np.uint8)
      mask = cv2.rectangle(mask, start_point, end_point, color, thickness)

      best_cut = np.where(mask==np.array([255, 255, 255]), frame, blurred_img)
      blurred_video[idx] = best_cut

    return cropped, blurred_video

### Teste Visual

In [ ]:
videos_test = pd.read_csv(PATH_TEST_RESIZED + "x_test_resized.csv")
videos_test = videos_test[['resized', 'label']].rename(columns={'resized': 'video_path'})
videos_test

,video_path,label
0,/content/drive/MyDrive/ucf_experiments/data/Pe...,1
1,/content/drive/MyDrive/ucf_experiments/data/Pe...,1
2,/content/drive/MyDrive/ucf_experiments/data/Pe...,1
3,/content/drive/MyDrive/ucf_experiments/data/Pe...,1
4,/content/drive/MyDrive/ucf_experiments/data/Pe...,1
...,...,...
196,/content/drive/MyDrive/ucf_experiments/data/Pe...,0
197,/content/drive/MyDrive/ucf_experiments/data/Pe...,0
198,/content/drive/MyDrive/ucf_experiments/data/Pe...,0
199,/content/drive/MyDrive/ucf_experiments/data/Pe...,0


In [ ]:

sample_url = videos_test.sample().video_path.squeeze()
sample_video = media.read_video(sample_url)

opt_frames, rgb_opt_frames = getOpticalFlow(sample_video, return_rgb_flow=True)

sample_url

'/content/drive/MyDrive/ucf_experiments/data/Peliculas/resized/newfi60_1_resized.mp4'

In [ ]:
rgb_cut, rgb_blur = dynamic_crop(opt_frames, sample_video)

In [ ]:
media.show_videos([sample_video, rgb_opt_frames, rgb_blur, rgb_cut])

This browser does not support the video tag.,This browser does not support the video tag.,This browser does not support the video tag.,This browser does not support the video tag.


## Spliting Optical Flow (Crop + Blur)

In [ ]:
#videos_teste = pd.read_csv(PATH_CROP + 'test.txt', names=['path', 'label'], sep=';')
#splitted = np.array_split(videos_teste, 10)
print(videos_test.label.value_counts())

0    101
1    100
Name: label, dtype: int64


In [ ]:
splitted_test = np.array_split(videos_test, 10)
len(splitted_test)

10

In [ ]:
def process_videos(videos_df, save_blur_base, save_crop_base, only_humans=True):
  videos = []

  for idx, row in videos_df.iterrows():
    video_path = row['video_path']
    label = row['label']

    rgb_frames = media.read_video(video_path)
    opt_frames, rgb_opt_frames = getOpticalFlow(rgb_frames, return_rgb_flow=False)
    result_crop_frames = []
    result_blur_frames = []


    rgb_cut, rgb_blur = dynamic_crop(opt_frames, rgb_frames)

    if only_humans: 
      for rgb_frame, rgb_blur_frame in zip(rgb_cut, rgb_blur):
        pred = model(rgb_frame)
        detection_df  = pred.pandas().xyxy[0]
        has_person    = 'person' in detection_df.name.unique()
        if has_person:
          result_crop_frames.append(rgb_frame)
          result_blur_frames.append(rgb_blur_frame)
    else:
      result_crop_frames = rgb_cut
      result_blur_frames = rgb_blur

    if len(result_crop_frames) < 1:
      continue

    video_blur_name = video_path.split('/')[-1].split('.')[0] + '_blur.mp4'
    video_crop_name = video_path.split('/')[-1].split('.')[0] + '_crop.mp4'

    video_blur_output_path = os.path.join(save_blur_base, video_blur_name)
    video_crop_output_path = os.path.join(save_crop_base, video_crop_name)

    media.write_video(video_blur_output_path, np.array(result_blur_frames))
    media.write_video(video_crop_output_path, np.array(result_crop_frames))

    row = {
      'crop_path': video_crop_output_path,
      'blur_path': video_blur_output_path,
      'label': label
    }

    videos.append((video_blur_output_path, video_crop_output_path, label))

  return videos

In [ ]:
checkpoints = pd.DataFrame([])
start_idx = 0

for idx, split in tqdm(enumerate(splitted_test[start_idx:]), total=len(splitted_test) - start_idx):
  
  results = process_videos(
      split,
      save_blur_base=PATH_BLUR_TEST,
      save_crop_base=PATH_CROP_TEST,
      only_humans=False
  )

  checkpoints = checkpoints.append(results, ignore_index=True)
  checkpoints.to_csv(PATH_CHECKPOINTS_TEST+'{}_checkpoint.csv'.format(str(start_idx + idx)))
  sleep(5)

100%|██████████| 10/10 [05:38<00:00, 33.89s/it]


In [ ]:
test_parts = glob.glob(PATH_CHECKPOINTS_TEST + '*_checkpoint.csv')
print(len(test_parts))

10


In [ ]:
files_test = pd.concat([ pd.read_csv(path) for path in test_parts ])
files_test

,Unnamed: 0,0,1,2
0,0,/content/drive/MyDrive/ucf_experiments/data/Pe...,/content/drive/MyDrive/ucf_experiments/data/Pe...,1
1,1,/content/drive/MyDrive/ucf_experiments/data/Pe...,/content/drive/MyDrive/ucf_experiments/data/Pe...,1
2,2,/content/drive/MyDrive/ucf_experiments/data/Pe...,/content/drive/MyDrive/ucf_experiments/data/Pe...,1
3,3,/content/drive/MyDrive/ucf_experiments/data/Pe...,/content/drive/MyDrive/ucf_experiments/data/Pe...,1
4,4,/content/drive/MyDrive/ucf_experiments/data/Pe...,/content/drive/MyDrive/ucf_experiments/data/Pe...,1
...,...,...,...,...
196,196,/content/drive/MyDrive/ucf_experiments/data/Pe...,/content/drive/MyDrive/ucf_experiments/data/Pe...,0
197,197,/content/drive/MyDrive/ucf_experiments/data/Pe...,/content/drive/MyDrive/ucf_experiments/data/Pe...,0
198,198,/content/drive/MyDrive/ucf_experiments/data/Pe...,/content/drive/MyDrive/ucf_experiments/data/Pe...,0
199,199,/content/drive/MyDrive/ucf_experiments/data/Pe...,/content/drive/MyDrive/ucf_experiments/data/Pe...,0


In [ ]:
def processed_data(data_files):
  processed_data = data_files.drop_duplicates()
  processed_data = processed_data.drop(columns=['Unnamed: 0'])

  processed_data = processed_data.dropna()
  processed_data = processed_data.reset_index(drop=True)

  dynamic_blur = processed_data[['0', '2']]
  dynamic_crop = processed_data[['1', '2']]

  return dynamic_blur, dynamic_crop

In [ ]:
dynamic_blur_test, dynamic_crop_test = processed_data(files_test)
print(dynamic_blur_test.shape)
print(dynamic_crop_test.shape)

(201, 2)
(201, 2)


In [ ]:
dynamic_crop_test.head(2)['1'].to_list()

['/content/drive/MyDrive/ucf_experiments/data/Peliculas/data_crop/test/newfi69_1_resized_crop.mp4',
 '/content/drive/MyDrive/ucf_experiments/data/Peliculas/data_crop/test/newfi1_1_resized_crop.mp4']

In [ ]:
dynamic_crop_test.head(2)['2'].to_list()

[1, 1]

In [ ]:
dynamic_blur_test.to_csv(PATH_BLUR + 'test.txt', header=None, index=False, sep=';')
dynamic_crop_test.to_csv(PATH_CROP + 'test.txt', header=None, index=False, sep=';')

## Resized Pós Crop

**Apenas o Crop precisa desse processo, pois o blur já está nas dimensões corretas**

In [ ]:
# Load
videos_teste = pd.read_csv(PATH_CROP + 'test.txt', names=['path', 'label'], sep=';')
splitted = np.array_split(videos_teste, 10)
len(splitted)

10

In [ ]:
videos_teste.shape

(201, 2)

In [ ]:
videos_teste.head(2)

,path,label
0,/content/drive/MyDrive/ucf_experiments/data/Pe...,1
1,/content/drive/MyDrive/ucf_experiments/data/Pe...,1


In [ ]:
videos_teste.head(3).path.to_list()

['/content/drive/MyDrive/ucf_experiments/data/Peliculas/data_crop/test/newfi69_1_resized_crop.mp4',
 '/content/drive/MyDrive/ucf_experiments/data/Peliculas/data_crop/test/newfi1_1_resized_crop.mp4',
 '/content/drive/MyDrive/ucf_experiments/data/Peliculas/data_crop/test/newfi3_1_resized_crop.mp4']

In [ ]:
videos_teste.label.value_counts()

0    101
1    100
Name: label, dtype: int64

In [ ]:
def resized_videos(row):
  videos = []

  video_path = row['path']
  new_path = PATH_CROP_TEST_RESIZED + row['path'].split('/')[-1]

  rgb_frames = media.read_video(video_path)
  
  rgb_frames = media.to_float01(rgb_frames)

  sampled_video = tf.image.resize(rgb_frames, [224,224]).numpy()
  media.write_video(new_path, sampled_video)

  return new_path


checkpoints = pd.DataFrame([])
start_idx = 0
results = pd.DataFrame()

for idx, split in tqdm(enumerate(splitted[start_idx:]), total=len(splitted) - start_idx):
  split['new_path'] = split.swifter.apply(resized_videos, axis=1)

  # results.append(result, ignore_index=True)
  # checkpoints = checkpoints.append(results, ignore_index=True)
  split.to_csv(PATH_CHECKPOINTS_TEST_RESIZED+'{}_checkpoint.csv'.format(str(start_idx + idx)))
  sleep(5)

  0%|          | 0/10 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/21 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:15<02:20, 15.64s/it]

Pandas Apply:   0%|          | 0/20 [00:00<?, ?it/s]

 20%|██        | 2/10 [00:30<02:02, 15.30s/it]

...

 80%|████████  | 8/10 [02:00<00:29, 14.81s/it]

Pandas Apply:   0%|          | 0/20 [00:00<?, ?it/s]

 90%|█████████ | 9/10 [02:15<00:14, 14.76s/it]

Pandas Apply:   0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 10/10 [02:29<00:00, 14.99s/it]


In [ ]:
test_parts = glob.glob(PATH_CHECKPOINTS_TEST_RESIZED  +'*_checkpoint.csv')
print(len(test_parts))

10


In [ ]:
files_test = pd.concat([ pd.read_csv(path) for path in test_parts ])
files_test.shape

(201, 4)

In [ ]:
files_test.head(3)

,Unnamed: 0,path,label,new_path
0,0,/content/drive/MyDrive/ucf_experiments/data/Pe...,1,/content/drive/MyDrive/ucf_experiments/data/Pe...
1,1,/content/drive/MyDrive/ucf_experiments/data/Pe...,1,/content/drive/MyDrive/ucf_experiments/data/Pe...
2,2,/content/drive/MyDrive/ucf_experiments/data/Pe...,1,/content/drive/MyDrive/ucf_experiments/data/Pe...


In [ ]:
def processed_data_resized(data_files):
  processed_data = data_files.drop_duplicates()
  processed_data = processed_data.drop(columns=['Unnamed: 0'])

  processed_data = processed_data.dropna()
  processed_data = processed_data.reset_index(drop=True)

  dynamic_crop = processed_data[['new_path', 'label']]

  return dynamic_crop

In [ ]:
dynamic_crop_test_resized = processed_data_resized(files_test)
print(dynamic_crop_test_resized.shape)

(201, 2)


In [ ]:
from pathlib import Path
paths = sorted(Path(PATH_CROP).iterdir(), key=os.path.getmtime, reverse=True)
paths

[PosixPath('/content/drive/MyDrive/ucf_experiments/data/Peliculas/data_crop/test_resized'),
 PosixPath('/content/drive/MyDrive/ucf_experiments/data/Peliculas/data_crop/test.txt'),
 PosixPath('/content/drive/MyDrive/ucf_experiments/data/Peliculas/data_crop/test')]

In [ ]:
dynamic_crop_test_resized[['new_path', 'label']].to_csv(PATH_CROP + 'test_resized.txt', header=None, index=False, sep=';')

## Filtragem de quantidade mínima de frames

In [ ]:
#Blur
PATH_BLUR_TEST = PATH_BLUR + 'test.txt'

#Crop
PATH_CROP_TEST = PATH_CROP + 'test_resized.txt'

In [ ]:
def filter_small_videos(row):
  cap = cv2.VideoCapture(row['video_path'])
  totalframecount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  label = row['label']

  if totalframecount < 32:

    row['small'] = 1
  else:
    row['small'] = 0

  return row

tqdm.pandas()

### Blur 

In [ ]:
PATH_BLUR

'/content/drive/MyDrive/ucf_experiments/data/Peliculas/data_blur/'

In [ ]:
videos_teste = pd.read_csv(PATH_BLUR + 'test.txt', sep=';', names=['video_path', 'label'])
print(videos_teste.shape)
print(videos_teste.label.value_counts())


(201, 2)
0    101
1    100
Name: label, dtype: int64


In [ ]:
videos_teste = videos_teste.swifter.apply(filter_small_videos, axis=1)
videos_teste.to_csv(PATH_BLUR + 'test_vQtFrames.txt', header=None, index=False, sep=';')

Pandas Apply:   0%|          | 0/201 [00:00<?, ?it/s]

In [ ]:
df_videos_teste = pd.read_csv(PATH_BLUR + 'test_vQtFrames.txt', header=None, sep=';', names=['video_path', 'label', 'frames32'])
df_videos_teste.head()

,video_path,label,frames32
0,/content/drive/MyDrive/ucf_experiments/data/Pe...,1,0
1,/content/drive/MyDrive/ucf_experiments/data/Pe...,1,0
2,/content/drive/MyDrive/ucf_experiments/data/Pe...,1,0
3,/content/drive/MyDrive/ucf_experiments/data/Pe...,1,0
4,/content/drive/MyDrive/ucf_experiments/data/Pe...,1,0


In [ ]:
df_videos_teste.label.value_counts()

0    101
1    100
Name: label, dtype: int64

In [ ]:
videos = {}
for key, video in df_videos_teste.loc[df_videos_teste.frames32 == 1].sample(10).iterrows():
  cap = cv2.VideoCapture(video.video_path)
  print(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

ValueError: ignored

In [ ]:
videos = {}
for key, video in df_videos_teste.loc[df_videos_teste.frames32 == 0].sample(10).iterrows():
  cap = cv2.VideoCapture(video.video_path)
  print(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

50
50
50
42
50
50
42
42
50
50


In [ ]:
df_videos_teste_filtered = df_videos_teste.loc[df_videos_teste.frames32 == 0]
df_videos_teste_filtered.shape

(201, 3)

In [ ]:
df_videos_teste_filtered.label.value_counts()

0    101
1    100
Name: label, dtype: int64

In [ ]:
videos = {}
for key, video in df_videos_teste_filtered.sample(10).iterrows():
  videos[str(key)+'_'+str(video.label)] = media.read_video(video.video_path)

media.show_videos(videos)

168_0 This browser does not support the video tag.,181_0 This browser does not support the video tag.,192_0 This browser does not support the video tag.,183_0 This browser does not support the video tag.,194_0 This browser does not support the video tag.,54_1 This browser does not support the video tag.,114_0 This browser does not support the video tag.


In [ ]:
df_videos_teste_filtered[['video_path', 'label']].to_csv(PATH_BLUR + 'test_vFilter_min32frames.txt', header=None, index=False, sep=';')

### Crop

In [ ]:
PATH_CROP

'/content/drive/MyDrive/ucf_experiments/data/Peliculas/data_crop/'

In [ ]:
videos_teste = pd.read_csv(PATH_CROP + 'test_resized.txt', sep=';', names=['video_path', 'label'])
videos_teste

,video_path,label
0,/content/drive/MyDrive/ucf_experiments/data/Pe...,1
1,/content/drive/MyDrive/ucf_experiments/data/Pe...,1
2,/content/drive/MyDrive/ucf_experiments/data/Pe...,1
3,/content/drive/MyDrive/ucf_experiments/data/Pe...,1
4,/content/drive/MyDrive/ucf_experiments/data/Pe...,1
...,...,...
196,/content/drive/MyDrive/ucf_experiments/data/Pe...,0
197,/content/drive/MyDrive/ucf_experiments/data/Pe...,0
198,/content/drive/MyDrive/ucf_experiments/data/Pe...,0
199,/content/drive/MyDrive/ucf_experiments/data/Pe...,0


In [ ]:
print(videos_teste.shape)
print(videos_teste.label.value_counts())

(201, 2)
0    101
1    100
Name: label, dtype: int64


In [ ]:
videos_teste = pd.read_csv(PATH_CROP + 'test_resized.txt', sep=';', names=['video_path', 'label'])

videos_teste = videos_teste.swifter.apply(filter_small_videos, axis=1)

Pandas Apply:   0%|          | 0/201 [00:00<?, ?it/s]

In [ ]:
videos_teste.to_csv(PATH_CROP + 'test_vQtFrames.txt', header=None, index=False, sep=';')

## Validando Teste

In [ ]:
df_videos_teste = pd.read_csv(PATH_CROP + 'test_vQtFrames.txt', header=None, sep=';', names=['video_path', 'label', 'frames32'])
df_videos_teste.head()

,video_path,label,frames32
0,/content/drive/MyDrive/ucf_experiments/data/Pe...,1,0
1,/content/drive/MyDrive/ucf_experiments/data/Pe...,1,0
2,/content/drive/MyDrive/ucf_experiments/data/Pe...,1,0
3,/content/drive/MyDrive/ucf_experiments/data/Pe...,1,0
4,/content/drive/MyDrive/ucf_experiments/data/Pe...,1,0


In [ ]:
df_videos_teste.head().video_path.tolist()

['/content/drive/MyDrive/ucf_experiments/data/Peliculas/data_crop/test_resized/newfi69_1_resized_crop.mp4',
 '/content/drive/MyDrive/ucf_experiments/data/Peliculas/data_crop/test_resized/newfi1_1_resized_crop.mp4',
 '/content/drive/MyDrive/ucf_experiments/data/Peliculas/data_crop/test_resized/newfi3_1_resized_crop.mp4',
 '/content/drive/MyDrive/ucf_experiments/data/Peliculas/data_crop/test_resized/newfi6_1_resized_crop.mp4',
 '/content/drive/MyDrive/ucf_experiments/data/Peliculas/data_crop/test_resized/newfi7_1_resized_crop.mp4']

In [ ]:
df_videos_teste.label.value_counts()

0    101
1    100
Name: label, dtype: int64

In [ ]:
df_videos_teste.shape

(201, 3)

In [ ]:
videos = {}
for key, video in df_videos_teste.loc[df_videos_teste.frames32 == 1].sample(10).iterrows():
  cap = cv2.VideoCapture(video.video_path)
  print(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

ValueError: ignored

In [ ]:
videos = {}
for key, video in df_videos_teste.loc[df_videos_teste.frames32 == 0].sample(10).iterrows():
  cap = cv2.VideoCapture(video.video_path)
  print(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

50
50
50
50
50
50
42
60
50
60


In [ ]:
df_videos_teste_filtered = df_videos_teste.loc[df_videos_teste.frames32 == 0]
df_videos_teste_filtered.shape

(201, 3)

In [ ]:
df_videos_teste_filtered.label.value_counts()

0    101
1    100
Name: label, dtype: int64

In [ ]:
videos = {}
for key, video in df_videos_teste_filtered.sample(10).iterrows():
  videos[str(key)+'_'+str(video.label)] = media.read_video(video.video_path)

media.show_videos(videos)

13_1 This browser does not support the video tag.,69_1 This browser does not support the video tag.,126_0 This browser does not support the video tag.,130_0 This browser does not support the video tag.,36_1 This browser does not support the video tag.,155_0 This browser does not support the video tag.,8_1 This browser does not support the video tag.,34_1 This browser does not support the video tag.


In [ ]:
df_videos_teste_filtered[['video_path', 'label']].to_csv(PATH_CROP + 'test_vFilter_min32frames.txt', header=None, index=False, sep=';')

## Uniform Sample

In [ ]:
#Blur
PATH_BLUR_UNIFORM_SAMPLE_TEST = PATH_BLUR + 'uniform_samples/test/'
create_dir(PATH_BLUR_UNIFORM_SAMPLE_TEST)
PATH_BLUR_UNIFORM_SAMPLE_CHECK_TEST = PATH_BLUR + 'uniform_samples/checkpoints_test/'
create_dir(PATH_BLUR_UNIFORM_SAMPLE_CHECK_TEST)

#Crop
PATH_CROP_UNIFORM_SAMPLE_TEST = PATH_CROP + 'uniform_samples/test/'
create_dir(PATH_CROP_UNIFORM_SAMPLE_TEST)
PATH_CROP_UNIFORM_SAMPLE_CHECK_TEST = PATH_CROP + 'uniform_samples/checkpoints_test/'
create_dir(PATH_CROP_UNIFORM_SAMPLE_CHECK_TEST)

Directory  /content/drive/MyDrive/ucf_experiments/data/Peliculas/data_blur/uniform_samples/test/  Created 
Directory  /content/drive/MyDrive/ucf_experiments/data/Peliculas/data_blur/uniform_samples/checkpoints_test/  Created 
Directory  /content/drive/MyDrive/ucf_experiments/data/Peliculas/data_crop/uniform_samples/test/  Created 
Directory  /content/drive/MyDrive/ucf_experiments/data/Peliculas/data_crop/uniform_samples/checkpoints_test/  Created 


### Blur

In [ ]:
videos_us_test = pd.read_csv(PATH_BLUR + 'test_vFilter_min32frames.txt', sep=';', header=None, names=['video_path', 'label'])

def uniform_sampling_test(row):
    path_save=PATH_BLUR_UNIFORM_SAMPLE_TEST
    target_frames=64
    # get total frames of input video and calculate sampling interval
    video_path = row['video_path']

    video = media.read_video(video_path)[...,:3]
    video = media.to_float01(video)

    len_frames = int(len(video))
    interval = int(np.ceil(len_frames/target_frames))
    # init empty list for sampled video and 
    sampled_video = []
    for i in range(0,len_frames,interval):
        sampled_video.append(video[i])     
    # calculate numer of padded frames and fix it 
    num_pad = target_frames - len(sampled_video)
    if num_pad>0:
        padding = [video[i] for i in range(-num_pad,0)]
        sampled_video += padding

            # get sampled video
    #sampled_video = tf.image.resize(sampled_video, [224,224])
    new_path = os.path.join(path_save, video_path.split('/')[-1])

    media.write_video(new_path, sampled_video)

    return new_path

In [ ]:
print(videos_us_test.shape)
display(videos_us_test.head(2))

(201, 2)


,video_path,label
0,/content/drive/MyDrive/ucf_experiments/data/Pe...,1
1,/content/drive/MyDrive/ucf_experiments/data/Pe...,1


In [ ]:
checkpoints = pd.DataFrame([])
start_idx = 0
results = pd.DataFrame()

splitted = np.array_split(videos_us_test, 10)

for idx, split in tqdm(enumerate(splitted[start_idx:]), total=len(splitted) - start_idx):
  split['new_path'] = split.apply(uniform_sampling_test, axis=1)

  # results.append(result, ignore_index=True)
  # checkpoints = checkpoints.append(results, ignore_index=True)
  split.to_csv(PATH_BLUR_UNIFORM_SAMPLE_CHECK_TEST+'{}_test_checkpoint.csv'.format(str(start_idx + idx)))
  sleep(5)

100%|██████████| 10/10 [02:20<00:00, 14.03s/it]


In [ ]:
print(PATH_BLUR_UNIFORM_SAMPLE_TEST)

/content/drive/MyDrive/ucf_experiments/data/Peliculas/data_blur/uniform_samples/test/


In [ ]:
test_parts = glob.glob(PATH_BLUR_UNIFORM_SAMPLE_CHECK_TEST + '*_checkpoint.csv')
files_test = pd.concat([ pd.read_csv(path) for path in test_parts ])
files_test.shape

(201, 4)

In [ ]:
files_test.head(2)

,Unnamed: 0,video_path,label,new_path
0,0,/content/drive/MyDrive/ucf_experiments/data/Pe...,1,/content/drive/MyDrive/ucf_experiments/data/Pe...
1,1,/content/drive/MyDrive/ucf_experiments/data/Pe...,1,/content/drive/MyDrive/ucf_experiments/data/Pe...


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [ ]:
files_test[['new_path', 'label']].to_csv(PATH_BLUR + 'test_uniform_sample.txt', header=None, index=False, sep=';')

### Crop

In [ ]:
videos_us_test = pd.read_csv(PATH_CROP + 'test_vFilter_min32frames.txt', sep=';', header=None, names=['video_path', 'label'])

def uniform_sampling_test(row):
    path_save=PATH_CROP_UNIFORM_SAMPLE_TEST
    target_frames=64
    # get total frames of input video and calculate sampling interval
    video_path = row['video_path']

    video = media.read_video(video_path)[...,:3]
    video = media.to_float01(video)

    len_frames = int(len(video))
    interval = int(np.ceil(len_frames/target_frames))
    # init empty list for sampled video and 
    sampled_video = []
    for i in range(0,len_frames,interval):
        sampled_video.append(video[i])     
    # calculate numer of padded frames and fix it 
    num_pad = target_frames - len(sampled_video)
    if num_pad>0:
        padding = [video[i] for i in range(-num_pad,0)]
        sampled_video += padding

            # get sampled video
    #sampled_video = tf.image.resize(sampled_video, [224,224])
    new_path = os.path.join(path_save, video_path.split('/')[-1])

    media.write_video(new_path, sampled_video)

    return new_path

In [ ]:
print(videos_us_test.shape)
display(videos_us_test.head(2))

(201, 2)


,video_path,label
0,/content/drive/MyDrive/ucf_experiments/data/Pe...,1
1,/content/drive/MyDrive/ucf_experiments/data/Pe...,1


In [ ]:
checkpoints = pd.DataFrame([])
start_idx = 0
results = pd.DataFrame()

splitted = np.array_split(videos_us_test, 10)

for idx, split in tqdm(enumerate(splitted[start_idx:]), total=len(splitted) - start_idx):
  split['new_path'] = split.apply(uniform_sampling_test, axis=1)

  # results.append(result, ignore_index=True)
  # checkpoints = checkpoints.append(results, ignore_index=True)
  split.to_csv(PATH_CROP_UNIFORM_SAMPLE_CHECK_TEST+'{}_test_checkpoint.csv'.format(str(start_idx + idx)))
  sleep(5)

100%|██████████| 10/10 [02:23<00:00, 14.35s/it]


In [ ]:
test_parts = glob.glob(PATH_CROP_UNIFORM_SAMPLE_CHECK_TEST + '*_checkpoint.csv')
files_test = pd.concat([ pd.read_csv(path) for path in test_parts ])
files_test.shape

(201, 4)

In [ ]:
files_test.head(2)

,Unnamed: 0,video_path,label,new_path
0,0,/content/drive/MyDrive/ucf_experiments/data/Pe...,1,/content/drive/MyDrive/ucf_experiments/data/Pe...
1,1,/content/drive/MyDrive/ucf_experiments/data/Pe...,1,/content/drive/MyDrive/ucf_experiments/data/Pe...


In [ ]:
files_test[['new_path', 'label']].to_csv(PATH_CROP + 'test_uniform_sample.txt', header=None, index=False, sep=';')

# Predições

## Imports

In [ ]:
import os
import glob
#from nvidia.dali import pipeline_def
#import nvidia.dali.fn as fn
#import nvidia.dali.types as types
#import nvidia.dali.plugin.tf as dali_tf
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf_v1
import logging
import numpy as np
#import cupy as cp
import mediapy
from matplotlib import pyplot as plt
import pandas as pd
from tqdm import tqdm
import cv2
from time import sleep
from time import time

from tensorflow.keras import Sequential
from tensorflow.keras.layers import ConvLSTM2D, Conv3D
from tensorflow.keras.layers import Reshape, AveragePooling2D 
from tensorflow.keras.layers import Dense, Dropout, Input 
from tensorflow.keras.layers import BatchNormalization 
from tensorflow.keras.layers import Flatten 
from tensorflow.keras.layers import MaxPooling3D
from tensorflow.keras.layers import LSTM, Multiply
from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow.keras.backend as K

from tensorflow import optimizers

tf.config.set_soft_device_placement(True)
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'false'
tf.get_logger().setLevel(logging.ERROR)


from keras.models import Sequential, Input, Model
from keras.models import load_model
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization, Activation, LeakyReLU, Add, Multiply
from keras.regularizers import l2
from keras.layers.core import Lambda
from keras.layers.core import Lambda
from tensorflow.keras.optimizers import Adam, SGD

## Crop - Sem Mixed Precision

In [ ]:
max_batch_size = 5
NUM_WORKERS=8
EPOCHS=60
BATCH_SIZE=8

N_CLASSES = 2
IMSIZE = (224, 224)
SequenceLength = 64

##STEPS_PER_EPOCH = int(len(train_dataset_list) / BATCH_SIZE)
#VAL_STEPS_PER_EPOCH = int(len(test_dataset_list) / BATCH_SIZE)

#shapes = ((BATCH_SIZE, SequenceLength, IMSIZE[0], IMSIZE[1], 3), (BATCH_SIZE, N_CLASSES))
#dtypes = (tf.float32, tf.float32)

def create_model(continue_training=False, last_checkpoint=0):
  metrics=[
    tf.keras.metrics.Precision(),
    tf.keras.metrics.CategoricalAccuracy(),
    tf.keras.metrics.Recall(),
    tf.keras.metrics.AUC()
  ]

  inputs = Input(shape=(SequenceLength, IMSIZE[0], IMSIZE[1], 3))

  #####################################################
  rgb = inputs
  rgb = Conv3D(
      16, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      16, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

  rgb = Conv3D(
      16, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      16, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

  rgb = Conv3D(
      32, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      32, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

  rgb = Conv3D(
      32, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      32, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

  #####################################################
  x = MaxPooling3D(pool_size=(8,1,1))(rgb)

  #####################################################
  x = Conv3D(
      64, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = Conv3D(
      64, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = MaxPooling3D(pool_size=(2,2,2))(x)

  x = Conv3D(
      64, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = Conv3D(
      64, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = MaxPooling3D(pool_size=(2,2,2))(x)

  x = Conv3D(
      128, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = Conv3D(
      128, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = MaxPooling3D(pool_size=(2,3,3))(x)

  #####################################################
  x = Flatten()(x)
  x = Dense(128,activation='relu')(x)
  x = Dropout(0.2)(x)
  x = Dense(32, activation='relu')(x)
  pred = Dense(N_CLASSES, activation='softmax')(x)
  model = Model(inputs=inputs, outputs=pred)

  sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)

  model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=metrics)

  return model

In [ ]:
import tensorflow.keras as keras
from tensorflow.keras.utils import Sequence
from keras.utils import np_utils
import mediapy as media

class DataGenerator(Sequence):
    """Data Generator inherited from keras.utils.Sequence
    Args: 
        directory: the path of data set, and each sub-folder will be assigned to one class
        batch_size: the number of data points in each batch
        shuffle: whether to shuffle the data per epoch
    Note:
        If you want to load file with other data format, please fix the method of "load_data" as you want
    """
    def __init__(self, input_file, batch_size=1, shuffle=True, data_augmentation=True, uniform_sample=False):
        # Initialize the params
        self.batch_size = batch_size
        self.input_file = input_file
        self.classes = 2
        self.shuffle = shuffle
        self.data_aug = data_augmentation
        self.uniform_sample = uniform_sample
        # Load all the save_path of files, and create a dictionary that save the pair of "data:label"
        self.X_path, self.Y_dict = self.search_data() 
        # Print basic statistics information
        self.print_stats()
        return None
        
    def search_data_v2(self):
        X_path = []
        Y_dict = {}

        df_videos = pd.read_csv(self.input_file, header=None, sep=';', names=['video_path', 'label'])

        X_path = df_videos[['video_path', 'label']].video_path.to_list()
        Y_dict = pd.get_dummies(df_videos[['video_path', 'label']], columns=['label']).set_index('video_path').T.to_dict(orient="list")

        return X_path, Y_dict

    def search_data(self):
        X_path = []
        Y_dict = {}
        one_hots = np_utils.to_categorical(range(self.classes))

        with open(self.input_file, 'r') as data_file:
           for line in data_file:
              line = line.rstrip('\n').split(';')
              path = line[0]

              label = int(line[1])
              X_path.append(path)
              Y_dict[path] = one_hots[label]

        return X_path, Y_dict
    
    def print_stats(self):
        # calculate basic information
        self.n_files = len(self.X_path)
        self.n_classes = self.classes
        self.indexes = np.arange(len(self.X_path))
        np.random.shuffle(self.indexes)
        # Output states
        print("Found {} files belonging to {} classes.".format(self.n_files,self.n_classes))
        for i,label in enumerate(range(self.classes)):
            print('%10s : '%(label),i)
        return None
    
    def __len__(self):
        # calculate the iterations of each epoch
        steps_per_epoch = np.ceil(len(self.X_path) / float(self.batch_size))
        return int(steps_per_epoch)

    def __getitem__(self, index):
        """Get the data of each batch
        """
        # get the indexs of each batch
        batch_indexs = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # using batch_indexs to get path of current batch
        batch_path = [self.X_path[k] for k in batch_indexs]
        # get batch data
        batch_x, batch_y = self.data_generation(batch_path)
        return batch_x, batch_y

    def on_epoch_end(self):
        # shuffle the data at each end of epoch
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def data_generation(self, batch_path):
        # load data into memory, you can change the np.load to any method you want
        batch_x = [self.load_data(x) for x in batch_path]
        batch_y = [self.Y_dict[x] for x in batch_path]
        # transfer the data format and take one-hot coding for labels
        batch_x = np.array(batch_x)
        batch_y = np.array(batch_y)
        return batch_x, batch_y
      
    def normalize(self, data):
        mean = np.mean(data)
        std = np.std(data)
        return (data-mean) / std
    
    def random_flip(self, video, prob):
        s = np.random.rand()
        if s < prob:
            video = np.flip(m=video, axis=2)
        return video    
    
    def uniform_sampling(self, video, target_frames=64):
        # get total frames of input video and calculate sampling interval 
        len_frames = int(len(video))
        interval = int(np.ceil(len_frames/target_frames))
        # init empty list for sampled video and 
        sampled_video = []
        for i in range(0,len_frames,interval):
            sampled_video.append(video[i])     
        # calculate numer of padded frames and fix it 
        num_pad = target_frames - len(sampled_video)
        if num_pad>0:
            padding = [video[i] for i in range(-num_pad,0)]
            sampled_video += padding     
        # get sampled video

        #sampled_video = tf.image.resize(sampled_video, [224,224])
        return np.array(sampled_video, dtype=np.float32)
    
    def dynamic_crop(self, video):
        # extract layer of optical flow from video
        opt_flows = video[...,3]
        # sum of optical flow magnitude of individual frame
        magnitude = np.sum(opt_flows, axis=0)
        # filter slight noise by threshold 
        thresh = np.mean(magnitude)
        magnitude[magnitude<thresh] = 0
        # calculate center of gravity of magnitude map and adding 0.001 to avoid empty value
        x_pdf = np.sum(magnitude, axis=1) + 0.001
        y_pdf = np.sum(magnitude, axis=0) + 0.001
        # normalize PDF of x and y so that the sum of probs = 1
        x_pdf /= np.sum(x_pdf)
        y_pdf /= np.sum(y_pdf)
        # randomly choose some candidates for x and y 
        x_points = np.random.choice(a=np.arange(224), size=10, replace=True, p=x_pdf)
        y_points = np.random.choice(a=np.arange(224), size=10, replace=True, p=y_pdf)
        # get the mean of x and y coordinates for better robustness
        x = int(np.mean(x_points))
        y = int(np.mean(y_points))
        # avoid to beyond boundaries of array
        x = max(56,min(x,167))
        y = max(56,min(y,167))
        # get cropped video 
        return video[:,x-56:x+56,y-56:y+56,:]  
    
    def color_jitter(self,video):
        # range of s-component: 0-1
        # range of v component: 0-255
        s_jitter = np.random.uniform(-0.2,0.2)
        v_jitter = np.random.uniform(-30,30)
        for i in range(len(video)):
            hsv = cv2.cvtColor(video[i], cv2.COLOR_RGB2HSV)
            s = hsv[...,1] + s_jitter
            v = hsv[...,2] + v_jitter
            s[s<0] = 0
            s[s>1] = 1
            v[v<0] = 0
            v[v>255] = 255
            hsv[...,1] = s
            hsv[...,2] = v
            video[i] = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
        return video
        
    def load_data(self, path):
        data = media.read_video(path)[...,:3]
        data = np.float32(data)
        # sampling 64 frames uniformly from the entire video

        if self.uniform_sample:
          data = self.uniform_sampling(video=data, target_frames=64)
        else:
          data = np.array(data, dtype=np.float32)

        # whether to utilize the data augmentation
        if  self.data_aug:
            data = self.color_jitter(data)
            data = self.random_flip(data, prob=0.5)
        # normalize

        data = self.normalize(data)
        return data

In [ ]:
df_test = DataGenerator(input_file=PATH_CROP + 'test_uniform_sample.txt',
                                batch_size=BATCH_SIZE, 
                                data_augmentation=False,
                                uniform_sample=False)
df_test

Found 201 files belonging to 2 classes.
         0 :  0
         1 :  1


In [ ]:
def generate_predicts(data, model, path_weights, name_csv):
  df_results = pd.DataFrame()

  with tf.device("/gpu:0"):
    model = model
    model.load_weights(path_weights)
    
    for idx, payload in tqdm(enumerate(data)):

      items, labels = payload
      predictions = model.predict_on_batch(items)
      K.clear_session()

      batch = {
          'true':np.argmax(predictions, axis=1),
          'predictions':np.argmax(labels, axis=1)
      }

      _temp = pd.DataFrame(batch)

      df_results = df_results.append(_temp, ignore_index=True)
      if idx % 100 == 0:
        print(idx)
        
      if idx == 1000:
        break

  df_results.to_csv(PATH_DATA + name_csv)

  report = classification_report(y_pred = df_results['predictions'], y_true=df_results['true'])

  return df_results, report

In [ ]:
PATH_DATA

'/content/drive/MyDrive/ucf_experiments/data/Peliculas/'

In [ ]:
best_epoch = 30
base = 'Movies'
version = 'vBaseline'
name_csv = 'Predicts-'+base+'-'+version+'-epoch-{}.csv'.format(str(best_epoch))
path_weights = '/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/model_vBaseline - rwf/checkpoints/model.{}.h5'.format(str(best_epoch))
df_results, report = generate_predicts(df_test, create_model(), path_weights, name_csv)
display(df_results.head(3))
print(report)

1it [00:21, 21.64s/it]

0


26it [03:39,  8.45s/it]


,true,predictions
0,0,0
1,1,1
2,0,0


              precision    recall  f1-score   support

           0       0.96      0.66      0.78       148
           1       0.49      0.92      0.64        53

    accuracy                           0.73       201
   macro avg       0.73      0.79      0.71       201
weighted avg       0.84      0.73      0.74       201



### The Best Epoch

In [ ]:
best_epoch = 16
base = 'Movies'
version = 'vBaseline'
name_csv = 'Predicts-'+base+'-'+version+'-epoch-{}.csv'.format(str(best_epoch))
path_weights = '/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/model_vBaseline - rwf/checkpoints/model.{}.h5'.format(str(best_epoch))
df_results, report = generate_predicts(df_test, create_model(), path_weights, name_csv)
display(df_results.head(3))
print(report)

1it [00:03,  3.21s/it]

0


26it [01:19,  3.06s/it]


,true,predictions
0,0,0
1,1,1
2,0,0


              precision    recall  f1-score   support

           0       0.98      0.69      0.81       144
           1       0.55      0.96      0.70        57

    accuracy                           0.77       201
   macro avg       0.77      0.83      0.75       201
weighted avg       0.86      0.77      0.78       201



In [7]:
best_epoch = 16
base = 'Movies'
version = 'vBaseline'
name_csv = 'Predicts-'+base+'-'+version+'-epoch-{}.csv'.format(str(best_epoch))
df_results = pd.read_csv(PATH_DATA + name_csv)
report = classification_report(y_pred = df_results['predictions'], y_true=df_results['true'], digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.9802    0.6875    0.8082       144
           1     0.5500    0.9649    0.7006        57

    accuracy                         0.7662       201
   macro avg     0.7651    0.8262    0.7544       201
weighted avg     0.8582    0.7662    0.7777       201



## Crop - Com Mixed Precision

In [ ]:
from tensorflow.keras import mixed_precision

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

In [ ]:
max_batch_size = 5
NUM_WORKERS=8
EPOCHS=60
BATCH_SIZE=8

N_CLASSES = 2
IMSIZE = (224, 224)
SequenceLength = 64

def create_model2(continue_training=False, last_checkpoint=0):
  metrics=[
    tf.keras.metrics.Precision(),
    tf.keras.metrics.CategoricalAccuracy(),
    tf.keras.metrics.Recall(),
    tf.keras.metrics.AUC()
  ]

  inputs = Input(shape=(SequenceLength, IMSIZE[0], IMSIZE[1], 3))

  #####################################################
  rgb = inputs
  rgb = Conv3D(
      16, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      16, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

  rgb = Conv3D(
      16, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      16, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

  rgb = Conv3D(
      32, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      32, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

  rgb = Conv3D(
      32, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      32, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

  #####################################################
  x = MaxPooling3D(pool_size=(8,1,1))(rgb)

  #####################################################
  x = Conv3D(
      64, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = Conv3D(
      64, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = MaxPooling3D(pool_size=(2,2,2))(x)

  x = Conv3D(
      64, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = Conv3D(
      64, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = MaxPooling3D(pool_size=(2,2,2))(x)

  x = Conv3D(
      128, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = Conv3D(
      128, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = MaxPooling3D(pool_size=(2,3,3))(x)

  #####################################################
  x = Flatten()(x)
  x = Dense(128,activation='relu')(x)
  x = Dropout(0.2)(x)
  x = Dense(32, activation='relu')(x)
  pred = Dense(N_CLASSES, activation='softmax', dtype='float32')(x)
  model = Model(inputs=inputs, outputs=pred)

  sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)

  model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=metrics)

  return model

In [ ]:
best_epoch = 30
base = 'Movies'
version = 'vBaseline_mixed_precision'
name_csv = 'Predicts-'+base+'-'+version+'-epoch-{}.csv'.format(str(best_epoch))
path_weights = '/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/model_vBaseline_mixed_precision - rwf/checkpoints/model.{}.h5'.format(str(best_epoch))
df_results, report = generate_predicts(df_test, create_model2(), path_weights, name_csv)
display(df_results.head(3))
print(report)

1it [00:03,  3.65s/it]

0


26it [01:21,  3.14s/it]


,true,predictions
0,0,0
1,1,1
2,0,0


              precision    recall  f1-score   support

           0       0.82      0.78      0.80       107
           1       0.76      0.81      0.78        94

    accuracy                           0.79       201
   macro avg       0.79      0.79      0.79       201
weighted avg       0.79      0.79      0.79       201



### The Best Epoch

In [ ]:
best_epoch = 13
base = 'Movies'
version = 'vBaseline_mixed_precision'
name_csv = 'Predicts-'+base+'-'+version+'-epoch-{}.csv'.format(str(best_epoch))
path_weights = '/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/model_vBaseline_mixed_precision - rwf/checkpoints/model.{}.h5'.format(str(best_epoch))
df_results, report = generate_predicts(df_test, create_model2(), path_weights, name_csv)
display(df_results.head(3))
print(report)

1it [00:03,  3.40s/it]

0


26it [01:21,  3.13s/it]


,true,predictions
0,0,0
1,0,1
2,0,0


              precision    recall  f1-score   support

           0       0.98      0.71      0.82       140
           1       0.59      0.97      0.73        61

    accuracy                           0.79       201
   macro avg       0.79      0.84      0.78       201
weighted avg       0.86      0.79      0.79       201



In [8]:
best_epoch = 13
base = 'Movies'
version = 'vBaseline_mixed_precision'
name_csv = 'Predicts-'+base+'-'+version+'-epoch-{}.csv'.format(str(best_epoch))
df_results = pd.read_csv(PATH_DATA + name_csv)
report = classification_report(y_pred = df_results['predictions'], y_true=df_results['true'], digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.9802    0.7071    0.8216       140
           1     0.5900    0.9672    0.7329        61

    accuracy                         0.7861       201
   macro avg     0.7851    0.8372    0.7772       201
weighted avg     0.8618    0.7861    0.7947       201



## Blur - Com Mixed Precision

In [ ]:
df_test = DataGenerator(input_file=PATH_BLUR + 'test_uniform_sample.txt',
                                batch_size=BATCH_SIZE, 
                                data_augmentation=False,
                                uniform_sample=False)
df_test

Found 201 files belonging to 2 classes.
         0 :  0
         1 :  1


In [ ]:
best_epoch = 30
base = 'Movies'
version = 'vBlur'
name_csv = 'Predicts-'+base+'-'+version+'-epoch-{}.csv'.format(str(best_epoch))
path_weights = '/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur/model mixed precision - rwf/checkpoints/model.{}.h5'.format(str(best_epoch))
df_results, report = generate_predicts(df_test, create_model2(), path_weights, name_csv)
display(df_results.head(3))
print(report)

1it [00:10, 10.35s/it]

0


26it [03:27,  7.97s/it]


,true,predictions
0,0,0
1,0,0
2,0,0


              precision    recall  f1-score   support

           0       0.96      0.79      0.87       123
           1       0.74      0.95      0.83        78

    accuracy                           0.85       201
   macro avg       0.85      0.87      0.85       201
weighted avg       0.87      0.85      0.85       201



### The Best Epoch

In [ ]:
best_epoch = 20
base = 'Movies'
version = 'vBlur'
name_csv = 'Predicts-'+base+'-'+version+'-epoch-{}.csv'.format(str(best_epoch))
path_weights = '/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur/model mixed precision - rwf/checkpoints/model.{}.h5'.format(str(best_epoch))
df_results, report = generate_predicts(df_test, create_model2(), path_weights, name_csv)
display(df_results.head(3))
print(report)

1it [00:03,  3.35s/it]

0


26it [01:21,  3.12s/it]


,true,predictions
0,0,0
1,0,0
2,0,0


              precision    recall  f1-score   support

           0       0.99      0.70      0.82       142
           1       0.58      0.98      0.73        59

    accuracy                           0.79       201
   macro avg       0.79      0.84      0.78       201
weighted avg       0.87      0.79      0.80       201



In [9]:
best_epoch = 20
base = 'Movies'
version = 'vBlur'
name_csv = 'Predicts-'+base+'-'+version+'-epoch-{}.csv'.format(str(best_epoch))
df_results = pd.read_csv(PATH_DATA + name_csv)
report = classification_report(y_pred = df_results['predictions'], y_true=df_results['true'], digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.9901    0.7042    0.8230       142
           1     0.5800    0.9831    0.7296        59

    accuracy                         0.7861       201
   macro avg     0.7850    0.8436    0.7763       201
weighted avg     0.8697    0.7861    0.7956       201



# FIM

In [ ]:
FIM

## Load Data

In [ ]:
files_test.to_csv(PATH_BLUR + 'test_uniform_sample.txt', header=None, index=False, sep=';')video_path	label	new_path
files_test.to_csv(PATH_CROP + 'test_uniform_sample.txt', header=None, index=False, sep=';')




## Metrics

In [ ]:
validation_df = pd.DataFrame()

with tf.device("/gpu:0"):
  model = create_model()
  model.load_weights('/content/drive/MyDrive/ucf_experiments/experiment_3/data_crop/model/checkpoints/model.196.h5')
  
  for idx, payload in enumerate(test_dataset):


    items, labels = payload
    predictions = model.predict_on_batch(items)
    K.clear_session()

    batch = {
        'true':np.argmax(predictions, axis=1),
        'predictions':np.argmax(labels, axis=1)
    }

    _temp = pd.DataFrame(batch)

    validation_df = validation_df.append(_temp, ignore_index=True)
    if idx % 100 == 0:
      print(idx)
      
    if idx == 1000:
      break

validation_df.to_csv('/content/drive/MyDrive/ucf_experiments/experiment_3/data_crop/rwf_wy_dynamic_crop_conv3d_tf.csv')

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_pred = validation_df['predictions'], y_true=validation_df['true'])
print(report)